# OPTIMIZED CNN

In [1]:
#importing All Libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import glob
import os


# Setting Paths To Directories
TEST_DATA="F:/MARATHI CHARECTORPROJECTMINI/DATASET/TEST/"
TRAIN_DATA="F:/MARATHI CHARECTORPROJECTMINI/DATASET/TR/"
DATA="F:/MARATHI CHARECTORPROJECTMINI/DATASET/"


# Now Creating DataSet From Images From Folders
FOLDERS=os.listdir(DATA)
IMAGES=[]        #Storing Image Pixel Values
LABELS=[]        #Storing Image Labels
import cv2
count=0
labels=[]
labels_name=[]

for i in os.listdir(DATA):
    p=os.path.join(DATA,i)
    io=int(i)
    labels.append(io)
    labels_name.append(i)
    for j in os.listdir(p):
        pat=os.path.join(p,j)
        img=cv2.imread(pat)
        IMAGES.append(img)
        LABELS.append(io)
        
        
        
NoOfClasses=len(labels)
print("No Of Classes {}".format(NoOfClasses))

print(len(LABELS))
print(len(IMAGES))

print("Current Image Dataset Have Size {} and Current Labels Have Size {}".format(len(IMAGES),len(LABELS)))


IMAGES=np.array(IMAGES)
print(" Current Training Image Dataset Size Is {}".format(IMAGES.shape))

LABELS=np.array(LABELS)
print(" Current Training Image Label Dataset Size Is {}".format(LABELS.shape))

from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(IMAGES,LABELS,test_size=0.2)


X_train=X_train/255.0
X_test=X_test/255.0

print(" We Have {} Training Images and {} Training Labels ".format(X_train.shape,Y_train.shape))

print(" We Have {} Testing Images and {} Testing Labels ".format(X_test.shape,Y_test.shape))

No Of Classes 36
52611
52611
Current Image Dataset Have Size 52611 and Current Labels Have Size 52611
 Current Training Image Dataset Size Is (52611, 28, 28, 3)
 Current Training Image Label Dataset Size Is (52611,)
 We Have (42088, 28, 28, 3) Training Images and (42088,) Training Labels 
 We Have (10523, 28, 28, 3) Testing Images and (10523,) Testing Labels 


In [3]:
#Defining Model For Hyperparameter Optimization..

def build_model(hp):  
    model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,3)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(NoOfClasses, activation='softmax')
    ])
  
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
    return model

In [4]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [5]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="MARATHI_CHAR_RGB")

INFO:tensorflow:Reloading Oracle from existing project output\MARATHI_CHAR_RGB\oracle.json
INFO:tensorflow:Reloading Tuner from output\MARATHI_CHAR_RGB\tuner0.json


In [6]:
tuner_search.search(X_train,Y_train,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [7]:
model=tuner_search.get_best_models(num_models=1)[0]

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       3136      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        48432     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1327152   
_________________________________________________________________
dense_1 (Dense)              (None, 36)                1764      
Total params: 1,380,484
Trainable params: 1,380,484
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(X_train,Y_train, epochs=3, validation_split=0.1)

Epoch 1/3
 186/1184 [===>..........................] - ETA: 0s - loss: 0.0964 - accuracy: 0.96 - ETA: 1:43 - loss: 0.0595 - accuracy: 0.98 - ETA: 2:22 - loss: 0.0403 - accuracy: 0.98 - ETA: 2:31 - loss: 0.0363 - accuracy: 0.98 - ETA: 2:35 - loss: 0.0296 - accuracy: 0.98 - ETA: 2:40 - loss: 0.0304 - accuracy: 0.98 - ETA: 2:41 - loss: 0.0268 - accuracy: 0.99 - ETA: 2:43 - loss: 0.0287 - accuracy: 0.98 - ETA: 2:48 - loss: 0.0262 - accuracy: 0.98 - ETA: 2:47 - loss: 0.0251 - accuracy: 0.99 - ETA: 2:48 - loss: 0.0487 - accuracy: 0.98 - ETA: 2:49 - loss: 0.0462 - accuracy: 0.98 - ETA: 2:49 - loss: 0.0436 - accuracy: 0.99 - ETA: 2:51 - loss: 0.0415 - accuracy: 0.99 - ETA: 2:49 - loss: 0.0394 - accuracy: 0.99 - ETA: 2:50 - loss: 0.0423 - accuracy: 0.99 - ETA: 2:51 - loss: 0.0398 - accuracy: 0.99 - ETA: 2:50 - loss: 0.0379 - accuracy: 0.99 - ETA: 2:51 - loss: 0.0367 - accuracy: 0.99 - ETA: 2:50 - loss: 0.0361 - accuracy: 0.99 - ETA: 2:50 - loss: 0.0348 - accuracy: 0.99 - ETA: 2:49 - loss: 0.036

In [20]:
SAVE_PATH="F:/MARATHI CHARECTORPROJECTMINI/"
import tensorflow as tf
from keras.models import load_model
model.save(os.path.join(SAVE_PATH,"RGB_HYPERPARAMETERIZED_IMAGESIZE_16_07_2020.h5"))

# .H5 to .tflite

In [21]:
import tensorflow as tf
model=tf.keras.models.load_model("F:/MARATHI CHARECTORPROJECTMINI/RGB_HYPERPARAMETERIZED_IMAGESIZE_16_07_2020.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()
open("F:/MARATHI CHARECTORPROJECTMINI/RGB_HYPERPARAMETERIZED_IMAGESIZE_16_07_2020.tflite", "wb").write(tflite_model) 

5524324

In [22]:
with open("F:/MARATHI CHARECTORPROJECTMINI/RGB_HYPERPARAMETERIZED_IMAGESIZE_28_1_06_2020.txt", 'w') as f:
    for i in labels:
        f.write(str(i))
        f.write("\n")

In [23]:
import tensorflow as tf
model=tf.keras.models.load_model("F:/MARATHI CHARECTORPROJECTMINI/RGB_HYPERPARAMETERIZED_IMAGESIZE_16_07_2020.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()
open("F:/MARATHI CHARECTORPROJECTMINI/ModelUpload.tflite", "wb").write(tflite_model) 

5524324

In [24]:
with open("F:/MARATHI CHARECTORPROJECTMINI/HYPERPARAMETERIZED_IMAGESIZE_labals.txt", 'w') as f:
    for i in labels:
        f.write(str(i))
        f.write("\n")

# NOW TESTING OUR MODEL

In [25]:
import numpy as np
from keras.preprocessing import image
import tensorflow as tf
from keras.models import load_model
import cv2
SAVE_PATH="F:/MARATHI CHARECTORPROJECTMINI/"
import os
import keras
import tensorflow as tf 
model=tf.keras.models.load_model("F:/MARATHI CHARECTORPROJECTMINI/HYPERPARAMETERIZED RGB/RGB_HYPERPARAMETERIZED_IMAGESIZE_28_1_06_2020.h5")


In [26]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       3136      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        48432     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1327152   
_________________________________________________________________
dense_1 (Dense)              (None, 36)                1764      
Total params: 1,380,484
Trainable params: 1,380,484
Non-trainable params: 0
_________________________________________________________________
None
